In [1]:
from datascience import *
import numpy as np
import re
from re import search
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [2]:
# scrape_table function from video demo
import pandas as pd
def scrape_table(table):
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    return Table.from_df(df)

In [22]:
r = requests.get("https://www.numbeo.com/cost-of-living/prices_by_country.jsp?itemId=6&regionCode=019&displayCurrency=CAD")
coke_text = BeautifulSoup(r.text, "html.parser")
coke_text_table = coke_text.find_all("table")[1]
coke_table = scrape_table(coke_text_table).drop("Rank").exclude(28)
coke_table

Country,Coke/Pepsi (0.33 liter bottle)
Kazakhstan,0.78
Paraguay,1.18
Portugal,1.96
Greece,2.53
Latvia,1.56
Iran,1.16
Morocco,0.63
Panama,2.04
Guatemala,1.38
Iraq,0.58


In [11]:
def convert(cur_from, cur_to, amount):
    payload = {
        'from': cur_from,
        'to': cur_to,
        'amount': amount,
        'api_key': '0a5659758c74b6dd01f96fdbc8fab87d'
    }
    r = requests.get('https://api.currencyscoop.com/v1/convert', params = payload)
    currency = r.json()
    return currency['response']['value']

In [21]:
#stolen shamlessly from example code
currency_url = "https://currencyscoop.com/supported-currencies"
currency_request = requests.get(currency_url)
currency_soup = BeautifulSoup(currency_request.text, 'html.parser')
currency_table = currency_soup.find("table")
currency_codes = scrape_table(currency_table)

codes = make_array()
for each_country in currency_codes.column("Currency"):
    codes = np.append(codes, each_country[-3:])

currency_codes = currency_codes.with_column("Code", codes)

def strip_code(country):
    return country[:-3]

currency_codes["Currency"] = currency_codes.apply(strip_code, "Currency")